In [1]:
import pandas as pd
import nltk
import spacy
from nltk import word_tokenize
from nltk.corpus import wordnet
import time
collision_data_preventable= pd.read_csv("../data/Clean_data/Collision_preventable.csv")
claim_data =  pd.read_csv("../data/TransLink Raw Data/claim_vehicle_employee_line.csv")

/home/simardeep/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,21) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [120]:
Collision_preventable = pd.read_excel('../data/TransLink Raw Data/2020 Collisions- Preventable and Non Preventable UBC Set Without Claim Number.xlsx', skiprows=  3)

176

In [5]:
count_df =  collision_data_preventable.groupby(["APTA", "Claim Desc."]).size().reset_index().rename(columns={0:"Count"})

In [6]:
count_df.sort_values(by=['Count'], ascending=False)

,APTA,Claim Desc.,Count
13,CO,BUS RF MIRROR HIT POLE,9
26,CO,BUS RR SIDE HIT POST,3
14,CO,BUS RF MIRROR HIT TREE,3
25,CO,BUS RR SIDE HIT POLE,2
85,CV,BUS RR SIDE HIT PARKED TP LF SIDE,2
...,...,...,...
57,CV,BUS 2131 HIT PARKED TP VEH,1
58,CV,BUS ALLEGEDLY CONTACTED TP RR CORNER - NO DMG,1
59,CV,BUS BIKE RACK REAR ENDED TP - NO DMG,1
60,CV,"BUS CHANGED LANES, RS REAR WHEEL HIT & CYCLIST...",1


In [7]:
# loading model of spacy
nlp= spacy.load("en_core_web_sm")

In [8]:
# loading all the stopwords
all_stopwords = nlp.Defaults.stop_words

In [9]:
# trail dataset for performing test
trial_df = count_df[['Claim Desc.']].iloc[:4,]


In [10]:
trial_df

,Claim Desc.
0,"9641 STRUCK POLE, BROKE RF MIRROR"
1,9780 HIT POLE WHILE PULLING INTO CURB
2,BROKE RIGHT FRONT MIRROR-UNSURE HOW
3,BUS DEWIRED AND DAMAGED TROLLEY POLE


In [111]:
chosen_verb_list = []
words =[ "bus", "hitting", "me"] 
for w in words:
        syns = wordnet.synsets(w)
        if syns:
            print(syns[0].lexname().split('.')[0])
#             if syns[0].lexname().split('.')[0] == 'verb':
#                 chosen_verb_list.append(w)
chosen_verb_list

    #print(w, syns[0].lexname().split('.')[0]) if syns else (w, None)

noun
noun
noun


[]

In [134]:
t = time.time()
list_of_list = list()
for sentence in count_df['Claim Desc.']:
    
    # declaring list of differrent pos tags
    intermediate_list = list()
    pos_list = list()
    noun_list = list()
    description_list = list()
    preposition_list = list()
    verb_list = list()
    adjective_list = list()
    # Removing the digits from the claim descriptions
    sentence = ''.join([i for i in sentence if not i.isdigit()])
    text = nltk.word_tokenize(sentence)
    
    #Removing stopwords
    
    text_without_sw = [word for word in text if (not word  in all_stopwords or  not word.isdigit())]
    result = ' '.join(text_without_sw)
    spacy_ready_text = nlp(result)
    
    # POS tagging
    for token in spacy_ready_text:
        pos_list.append(token.pos_)
        if token.pos_ == 'NOUN':
            noun_list.append(str(token))
        elif token.pos_ == 'VERB':
            verb_list.append(str(token))
        elif token.pos_ == 'PROPN':
            preposition_list.append(str(token))

    tag = 'unknown'        
    chosen_verb_list = list()
    if verb_list == [] and preposition_list != []:
        for w in preposition_list:
            syns = wordnet.synsets(w)
            if syns:
                if syns[0].lexname().split('.')[0] == 'verb':
                    #print(w)
                    chosen_verb_list.append(w)
                else:
                    continue
        if chosen_verb_list == []:
            chosen_verb_list = preposition_list
                    
    elif preposition_list == [] and verb_list != []:
        for w in verb_list:
            syns = wordnet.synsets(w)
            if syns:
                if syns[0].lexname().split('.')[0] == 'verb':
                    chosen_verb_list.append(w)
                else:
                    continue
        if chosen_verb_list == []:
            chosen_verb_list = verb_list            
                
                    
    elif verb_list == [] and preposition_list == []:
        chosen_verb_list.append(tag)
        
    else:
        for w in verb_list:
            syns = wordnet.synsets(w)
            if syns:
                if syns[0].lexname().split('.')[0] == 'verb':
                    chosen_verb_list.append(w)
                else:
                    continue
        if chosen_verb_list == [] and verb_list != []:
            chosen_verb_list = verb_list
            
        elif chosen_verb_list == [] and preposition_list != []:
            chosen_verb_list = preposition_list
        
    
    impact_list = []
    if noun_list:
        impact_list.append(noun_list[-1])

    
        

        
    intermediate_list.extend([sentence, pos_list,preposition_list,verb_list, chosen_verb_list, impact_list ])
    list_of_list.append(intermediate_list)    
pos_df = pd.DataFrame(list_of_list, columns=['Description','POS','preposition','verb', 'chosen_verb', 'impact'])

time.time() - t
    

1.1569974422454834

In [169]:
sum(list(pos_df['chosen_verb'].astype(bool)))

161

In [172]:
pd.concat([collision_data_preventable, pos_df['chosen_verb'], pos_df['impact']],axis = 1)

,Claim ID,Loss Date (MM/DD/YEAR),Time of Loss,Loss Location At,Loss Location On,City of Incident,Preventable/NonPreventable,Cost Centre,Bus Route Code,Vehicle Number,...,Description Code 1 Desc,APTA,APTA Desc,Claim Desc.,Asset Vehicle Year,Asset Manufacturer Model,Asset Manufacturer,Status,chosen_verb,impact
0,NaN,2020-03-26,847,Thurlow St,Dunsmuir St,Vancouver,P,167,022,18386,...,COLLISION WITH FIXED OBJECT,COG,Collisions with Objects (at G/C),TURNING BUS R/S HIT POLE,2018,DE40LF,NOV,OPEN,"[STRUCK, BROKE]",[RF]
1,NaN,2020-03-25,2125,NaN,Lonsdale Quay,North Vancouver,P,169,239,9608,...,NaN,CO,Collisions with Objects,"BUS UNDERCARRIAGE HIT CURB, TOW REQUIRED",2007,40LFC,NOV,OPEN,"[HIT, PULLING]",[CURB]
2,NaN,2020-03-24,1830,Earles St,W 41st Ave,Vancouver,P,167,41,18340,...,VEH APPROACH FROM ANGLE - LEFT,CV,Collisions with Other Vehicles,BUS RAN RED LIGHT AND LEFT FRONT CORNER HIT TP...,2018,DE40LF,NOV,OPEN,[BROKE],[FRONT]
3,NaN,2020-03-23,1127,Hastings st,Granville St,Vancouver,P,167,15,18377,...,RV STRIKES PARKED VEH,CV,Collisions with Other Vehicles,BUS RR SIDE STRUCK PARKED TP RR SIDE,2018,DE40LF,NOV,OPEN,"[BUS, DEWIRED]",[POLE]
4,NaN,2020-03-21,1155,Steeple Dr,Lansdowne Dr,Coquitlam,P,364,153,16546,...,AWARD REQUIRES FURTHER ACTION,CV,Collisions with Other Vehicles,SHUTTLE RF MIRROR CONTACTED PARKED TP LF MIRROR,2016,GS28CS,GM,OPEN,"[SIDE, WINDOW, LS, HIT]",[FENCE]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,NaN,2020-01-05,1215,W 41 Ave,Granville St,Vancouver,P,167,010,2559,...,RIGHT MIRROR FIXED OBJECT-PULLING INTO BUS STOP,CO,Collisions with Objects,BUS RF MIRROR HIT POLE,2008,E60LF,WF,OPEN,NaN,NaN
172,NaN,2020-01-05,1915,Langley St,Armstrong Ave,Burnaby,P,357,410,16011,...,RV STRIKES PARKED VEH,CV,Collisions with Other Vehicles,BUS RF CORNER HIT PARKED TP LF MIRROR - NO DMG,2016,XN40,WF,OPEN,NaN,NaN
173,NaN,2020-01-04,1046,Bute St,Robson St,Vancouver,P,167,005,2198,...,TRL/PL DEWIRE BRKN PL NOT OVRHD,CV,Collisions with Other Vehicles,BUS/BUS 2198 TROLLEY POLE CLIPPED STATIONARY 2...,2006,E40LF,WF,OPEN,NaN,NaN
174,NaN,2020-01-03,655,Oak St,Marine Dr,Vancouver,P,167,009,2208,...,TRL/PL DEWIRE BRKN PL NOT OVRHD,CV,Collisions with Other Vehicles,BUS/BUS 2208 TROLLEY POLE HIT STATIONARY 2543 ...,2006,E40LF,WF,FINAL,NaN,NaN


In [159]:
pos_df

,Description,POS,preposition,verb,chosen_verb,impact
0,"STRUCK POLE, BROKE RF MIRROR","[PROPN, PROPN, PUNCT, PROPN, NOUN, PROPN]","[STRUCK, POLE, BROKE, MIRROR]",[],"[STRUCK, BROKE]",[RF]
1,HIT POLE WHILE PULLING INTO CURB,"[VERB, NOUN, ADP, VERB, ADP, NOUN]",[],"[HIT, PULLING]","[HIT, PULLING]",[CURB]
2,BROKE RIGHT FRONT MIRROR-UNSURE HOW,"[PROPN, PROPN, NOUN, PROPN, PUNCT, PROPN, ADV]","[BROKE, RIGHT, MIRROR, UNSURE]",[],[BROKE],[FRONT]
3,BUS DEWIRED AND DAMAGED TROLLEY POLE,"[PROPN, PROPN, CCONJ, NOUN, NOUN, NOUN]","[BUS, DEWIRED]",[],"[BUS, DEWIRED]",[POLE]
4,BUS DRIVER SIDE WINDOW AND LS HIT FENCE,"[NOUN, NOUN, PROPN, PROPN, CCONJ, PROPN, PROPN...","[SIDE, WINDOW, LS, HIT]",[],"[SIDE, WINDOW, LS, HIT]",[FENCE]
...,...,...,...,...,...,...
156,BUS/BUS REAR BUMPER CONTACTED WINDSHIELD,"[PROPN, SYM, NOUN, NOUN, PROPN, ADJ, NOUN]","[BUS, BUMPER]",[],"[BUS, BUMPER]",[WINDSHIELD]
157,BUS/BUS REAR ENDED PARKED,"[PROPN, SYM, NOUN, ADJ, PROPN, PROPN]","[BUS, ENDED, PARKED]",[],"[ENDED, PARKED]",[BUS]
158,BUS/BUS RR SIDE CLIPPED STATIONARY LR CORNER,"[PROPN, SYM, PROPN, NOUN, PROPN, PROPN, PROPN,...","[BUS, BUS, SIDE, CLIPPED, STATIONARY, LR, CORNER]",[],[CLIPPED],[RR]
159,BUS/BUS- TROLLEY POLES HIT TROLLEY POLES,"[PROPN, SYM, PROPN, PROPN, NOUN, VERB, NOUN, N...","[BUS, BUS-, TROLLEY]",[HIT],[HIT],[POLES]


In [ ]:
## Do not run this, its 1.75 lakh rows
list_of_list = list()
for sentence in claim_clean:
    intermediate_list = list()
    pos_list = list()
    number_list = list()
    noun_list = list()
    description_list = list()
    preposition_list = list()
    verb_list = list()
    adjective_list = list()
    sentence = ''.join([i for i in sentence if not i.isdigit()])
    text = word_tokenize(sentence)
    text_without_sw = [word for word in text if (not word  in all_stopwords or  not word.isdigit())]
    result = ' '.join(text_without_sw)
    spacy_ready_text = nlp(result)
    for token in spacy_ready_text:
        pos_list.append(token.pos_)
        if token.pos_ == 'NUM':
            number_list.append(token)
        elif token.pos_ == 'NOUN':
            noun_list.append(token)
        elif token.pos_ == 'VERB':
            verb_list.append(token)
        elif token.pos_ == 'PROPN':
            preposition_list.append(token)
        elif token.pos_ == 'ADJ':
            adjective_list.append(token)

        
    intermediate_list.extend([sentence, pos_list, noun_list,number_list,verb_list, preposition_list, adjective_list ])
    list_of_list.append(intermediate_list)    
claim_pos_df = pd.DataFrame(list_of_list, columns=['Description','POS', 'Nouns', 'Numbers', 'Verbs', 'Preposition', 'Adjective'])
    

In [57]:
list_of_list = list()
for sentence in count_df['Claim Desc.']:
    intermediate_list = list()
    pos_list = list()
    noun_list = list()
    description_list = list()
    preposition_list = list()
    verb_list = list()
    adjective_list = list()
    sentence = ''.join([i for i in sentence if not i.isdigit()])
    text = nltk.word_tokenize(sentence)
    text_without_sw = [word for word in text if (not word  in all_stopwords or  not word.isdigit())]
    result = ' '.join(text_without_sw)
    spacy_ready_text = nlp(result)
    for token in spacy_ready_text:
        pos_list.append(token.pos_)
        if token.pos_ == 'NOUN':
            noun_list.append(token)
        elif token.pos_ == 'VERB':
            verb_list.append(token)
        elif token.pos_ == 'PROPN':
            preposition_list.append(token)
        elif token.pos_ == 'ADJ':
            adjective_list.append(token)
    
        
    intermediate_list.extend([sentence, pos_list, noun_list,verb_list, preposition_list, adjective_list ])
    list_of_list.append(intermediate_list)    
pos_df = pd.DataFrame(list_of_list, columns=['Description','POS', 'Nouns', 'Verbs', 'Preposition', 'Adjective'])
    

In [13]:
pos_df['Verbs']

,Description,POS,Nouns,Numbers,Verbs,Preposition,Adjective
0,"STRUCK POLE, BROKE RF MIRROR","[PROPN, PROPN, PUNCT, PROPN, NOUN, PROPN]",[RF],[],[],"[STRUCK, POLE, BROKE, MIRROR]",[]
1,HIT POLE WHILE PULLING INTO CURB,"[VERB, NOUN, ADP, VERB, ADP, NOUN]","[POLE, CURB]",[],"[HIT, PULLING]",[],[]
2,BROKE RIGHT FRONT MIRROR-UNSURE HOW,"[PROPN, PROPN, NOUN, PROPN, PUNCT, PROPN, ADV]",[FRONT],[],[],"[BROKE, RIGHT, MIRROR, UNSURE]",[]
3,BUS DEWIRED AND DAMAGED TROLLEY POLE,"[PROPN, PROPN, CCONJ, NOUN, NOUN, NOUN]","[DAMAGED, TROLLEY, POLE]",[],[],"[BUS, DEWIRED]",[]
4,BUS DRIVER SIDE WINDOW AND LS HIT FENCE,"[NOUN, NOUN, PROPN, PROPN, CCONJ, PROPN, PROPN...","[BUS, DRIVER, FENCE]",[],[],"[SIDE, WINDOW, LS, HIT]",[]
...,...,...,...,...,...,...,...
156,BUS/BUS REAR BUMPER CONTACTED WINDSHIELD,"[PROPN, SYM, NOUN, NOUN, PROPN, ADJ, NOUN]","[BUS, REAR, WINDSHIELD]",[],[],"[BUS, BUMPER]",[CONTACTED]
157,BUS/BUS REAR ENDED PARKED,"[PROPN, SYM, NOUN, ADJ, PROPN, PROPN]",[BUS],[],[],"[BUS, ENDED, PARKED]",[REAR]
158,BUS/BUS RR SIDE CLIPPED STATIONARY LR CORNER,"[PROPN, SYM, PROPN, NOUN, PROPN, PROPN, PROPN,...",[RR],[],[],"[BUS, BUS, SIDE, CLIPPED, STATIONARY, LR, CORNER]",[]
159,BUS/BUS- TROLLEY POLES HIT TROLLEY POLES,"[PROPN, SYM, PROPN, PROPN, NOUN, VERB, NOUN, N...","[POLES, TROLLEY, POLES]",[],[HIT],"[BUS, BUS-, TROLLEY]",[]


In [149]:
pos_df.to_excel('pos_data.xlsx', index=False)

In [143]:
doc = nlp("HIT POLE WHILE PULLING INTO CURB")
for token in doc:
    print(token.pos_)

VERB
NOUN
ADP
VERB
ADP
NOUN


In [ ]:
text = word_tokenize("BUS/BUS 9770 REAR ENDED PARKED 9648")
nltk.pos_tag(text)

In [145]:
s = '12 ab cd 405'
result = ''.join([i for i in s if not i.isdigit()])
result


' ab cd '

In [3]:
from nltk.corpus import wordnet
words =["BROKE", "RIGHT", "MIRROR", "UNSURE"]
for w in words:
    syns = wordnet.synsets(w)
    print(w, syns[0].lexname().split('.')[0]) if syns else (w, None)

NameError: name 'BROKE' is not defined

In [157]:
claim_clean = claim_data['claim_desc'].dropna()

In [159]:
claim_clean

0                                T/P REAR ENDED STOPPED BUS
1                          PASS TRIPPED BEFORE BOARDING BUS
2                              PASS HIT BAR ON W/CHAIR SEAT
3                                BUS MIRROR HIT PED ON HEAD
4                                   BUS HIT PARKED BUS 8040
                                ...                        
175918    PASS W/WALKER FELL BACKWARDS WHILE BOARDING RE...
175919           PASSING UHAUL CLIPPED BUS LF MIRROR-NO DMG
175920        U/K MALE FELL TO SIDEWALK WHILE ALIGHTING BUS
175921                        240 EB'D DUE TO VOBC AT HO IB
175922      LEFT TURNING TP RF CORNER HIT STATIONARY BUS LS
Name: claim_desc, Length: 174472, dtype: object